In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
import numpy as np
from sklearn.decomposition import PCA

In [2]:
# Load the training data
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [3]:
X = X.drop('sub_area', axis=1)
X_test = X_test.drop('sub_area', axis=1)
used.append('Removed sub_area')

In [4]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test) 
used.append('OneHot Encoding')

In [5]:
# from sklearn.calibration import LabelEncoder

# categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
# print("Train: Categorical columns:", categorical_columns)

# label_encoder = LabelEncoder()

# for column in categorical_columns:
#     X[column] = label_encoder.fit_transform(X[column])

# categorical_columns_test = X_test.select_dtypes(include=['object']).columns.tolist()
# print("Test: Categorical columns:", categorical_columns_test)

# label_encoder = LabelEncoder()

# for column in categorical_columns_test:
#     X_test[column] = label_encoder.fit_transform(X_test[column])

# used.append('Label Encoding')

In [6]:
# # drop all columns in X_train with dtypes object
# for col in X.columns:
#     if X[col].dtype == 'object':
#         X.drop(col, axis=1, inplace=True)

# # drop all columns in X_test with dtypes object
# for col in X_test.columns:
#     if X_test[col].dtype == 'object':
#         X_test.drop(col, axis=1, inplace=True)

# used.append("Removed Object Dtypes")

In [7]:
X = X.astype('float32')
X_test = X_test.astype('float32')
used.append("Converted All Columns To float32")

In [8]:
X = np.log1p(X)
X_test = np.log1p(X_test)
used.append('log Normalization')

In [9]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
used.append("StandardScaler")

In [11]:
X_train.shape

(127054, 287)

In [12]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2)#, interaction_only=True)
# X_train = poly.fit_transform(X_train)
# X_test = poly.fit_transform(X_test)
# used.append('PolynomialFeatures W/O Interaction')
# # used.append('PolynomialFeatures With Interaction')

In [13]:
# # save X_train to csv
# X_train.to_csv('train with poly w/o int.2.csv', index=False)
# X_test.to_csv('test with poly w/o int.csv', index=False)

In [14]:
# pca = PCA(n_components=200)
# principalComponents = pca.fit_transform(X_train)
# X_train = pd.DataFrame(data = principalComponents)

# pca2 = PCA(n_components=200)
# principalComponents = pca2.fit_transform(X_test)
# X_test = pd.DataFrame(data = principalComponents)

# used.append('PCA (n=200)')

<h1>With Keras<h1>

In [15]:
# features_used = X_train.shape[1]

# # optimizerUsing = SGD(learning_rate=0.01, momentum=0.9)

# # optimizerUsing = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
# optimizerUsing = Adam(lr=0.001)#, epsilon=1e-07)

# # Build the neural network
# os.environ["KERAS_BACKEND"] = "torch"
# model = Sequential()
# model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.3))  # You can use either dropout or early stopping
# model.add(Dense(68, activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.3))
# model.add(Dense(40, activation='relu', kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='linear'))  # Output layer with linear activation for regression

# # Compile the model
# model.compile(loss='mean_squared_error', optimizer=optimizerUsing)

# # Define early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=1800, validation_data=(X_val, y_val), callbacks=[early_stopping])

# used.append("Keras With Torch Backend")
# # used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# # used.append("2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, Hidden Layer 2 with 50 neurons relu activation, Dropout 0.3, Output linear")
# used.append("3 Hidden Layers: Hidden Layer 1 with 128 neurons relu activation L2 regularization, Hidden Layer 2 with 64 neurons relu activation L2 regularization, Hidden Layer 3 with 40 neurons relu activation L2 regularization")
# used.append("Output linear")
# used.append("Optimizer: Adam(lr=0.01)")
# used.append("Loss Calculation: Mean Squared Error")
# used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
# used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")
# # used.append("batch_size=1800, epochs=100, early_stopping")

# # Make predictions
# predictions = model.predict(X_test)

<h1>With Skilearn<h1>

In [16]:
features_used = X_train.shape[1]

# # optimizerUsing = SGD(learning_rate=0.01, momentum=0.9)

# # optimizerUsing = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
# optimizerUsing = Adam(lr=0.001)#, epsilon=1e-07)

# Build the neural network using MLPRegressor from scikit-learn
model = MLPRegressor(hidden_layer_sizes=(300,210,180,70,10), activation='relu', solver='adam', shuffle=False, max_iter=100,
                     batch_size=1800, early_stopping=True, verbose=True, random_state=42)

# Train the model
model.fit(X_train, y_train)

used.append("MLPRegressor(hidden_layer_sizes=(300,210,180,70,10), activation='relu', solver='adam', shuffle=False, max_iter=100, batch_size=1800, early_stopping=True, verbose=True, random_state=42)")
# used.append("1 Hidden Layer: Hidden Layer 1 with 140 neurons relu activation, Output linear")
# used.append("2 Hidden Layers: Hidden Layer 1 with 100 neurons relu activation, Hidden Layer 2 with 50 neurons relu activation, Dropout 0.3, Output linear")
used.append("5 Hidden Layers: Hidden Layer 1 with 300 neurons, Hidden Layer 2 with 210 neurons, Hidden Layer 3 with 180 neurons, Hidden Layer 4 with 70 neurons, Hidden Layer 5 with 10 neurons")
# used.append("Output linear")
used.append("Optimizer: Adam")
# used.append("Loss Calculation: Mean Squared Error")
# used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
# used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers only)")
used.append("batch_size=1800, epochs=100, early_stopping")

# Make predictions
predictions = model.predict(X_test)

Iteration 1, loss = 347250411246248.37500000
Validation score: -0.431759
Iteration 2, loss = 229930352120620.06250000
Validation score: 0.592161
Iteration 3, loss = 92953284825770.54687500
Validation score: 0.628651
Iteration 4, loss = 90452496585488.39062500
Validation score: 0.634392
Iteration 5, loss = 89449477407107.01562500
Validation score: 0.637943
Iteration 6, loss = 88789303984908.76562500
Validation score: 0.640337
Iteration 7, loss = 88314346877541.39062500
Validation score: 0.642081
Iteration 8, loss = 87949704385413.56250000
Validation score: 0.643417
Iteration 9, loss = 87657822830068.43750000
Validation score: 0.644487
Iteration 10, loss = 87410471174696.32812500
Validation score: 0.645401
Iteration 11, loss = 87189104212683.43750000
Validation score: 0.646214
Iteration 12, loss = 86985718130244.31250000
Validation score: 0.646953
Iteration 13, loss = 86795207514477.56250000
Validation score: 0.647635
Iteration 14, loss = 86614965577604.23437500
Validation score: 0.64826

In [17]:
# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('Day8.6.csv', index=False)

In [18]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 287

Removed sub_area
OneHot Encoding
Converted All Columns To float32
log Normalization
StandardScaler
MLPRegressor(hidden_layer_sizes=(300,210,180,70,10), activation='relu', solver='adam', shuffle=False, max_iter=100, batch_size=1800, early_stopping=True, verbose=True, random_state=42)
5 Hidden Layers: Hidden Layer 1 with 300 neurons, Hidden Layer 2 with 210 neurons, Hidden Layer 3 with 180 neurons, Hidden Layer 4 with 70 neurons, Hidden Layer 5 with 10 neurons
Optimizer: Adam
batch_size=1800, epochs=100, early_stopping
